**목표: Text-to-Image 모델 Latent Diffusion Model (LDM) 을 활용하여 텍스트 프롬프트에 따른 이미지 생성을 실습하고, 잠재적 표현의 변화에 따른 이미지 변화를 분석한다.** <br>
<br>
1. **환경 설정 및 LDM 로드**
   - 필요한 라이브러리(diffusers, transformers, accelerate, scipy, safetensors 등)를 설치하고 임포트한다.
   - Hugging Face Hub 에서 사전 훈련된 LDM 체크포인트(`"CompVis/ldm-text2im-large-256"`)를 로드한다.
   - 모델을 GPU 로 이동시킨다.

2. **기본 이미지 생성**
   - 간단한 텍스트 프롬프트(예: `"A painting of a squirrel eating a burger"`)를 사용하여 이미지를 생성한다.
   - 생성된 이미지를 시각화하고 저장한다.

3. **잠재적 표현(Latent Representation)의 변화 관찰**
   - **프롬프트 설정**: 두 개의 서로 다른 텍스트 프롬프트(`prompt_1`, `prompt_2`)를 설정한다. (예: `prompt_1 = "A cat sitting on a park bench"`, `prompt_2 = "A dog running in a field of flowers"`)
   - **텍스트 임베딩**: 설정된 두 프롬프트를 LDM 의 텍스트 인코더를 사용하여 각각 텍스트 임베딩(`text_embeddings_1`, `text_embeddings_2`)으로 변환한다.
   - **선형 보간(Linear Interpolation)**:
     - 두 텍스트 임베딩 사이를 선형으로 보간하여 여러 개의 중간 임베딩을 생성한다. (예: `alpha` 값을 0부터 1까지 변화시키며 `(1-alpha)*text_embeddings_1 + alpha*text_embeddings_2` 계산)
     - 보간된 각 임베딩을 사용하여 이미지를 생성한다. (`guidance_scale`, `num_inference_steps` 등의 파라미터는 동일하게 유지)
   - **결과 분석 및 시각화**:
     - 생성된 일련의 이미지들을 순서대로 시각화한다.
     - `alpha` 값의 변화에 따라 이미지의 어떤 특성(객체, 배경, 스타일 등)이 점진적으로 변하는지 관찰하고 기록한다.
     - 두 프롬프트의 주요 의미 요소가 이미지 상에서 어떻게 혼합되고 전환되는지 분석한다.

4. **실험 파라미터 조정 (선택 사항)**
   - `guidance_scale`, `num_inference_steps`, `num_images_per_prompt` 등의 파라미터를 변경해보며 결과 이미지의 품질과 다양성에 미치는 영향을 관찰한다.
   - 보간 스텝 수(`interpolation_steps`)를 조절하여 더 부드럽거나 급격한 변화를 유도해본다.

5. **결과 정리 및 보고**
   - 실험 과정, 설정한 프롬프트, 사용한 파라미터, 생성된 주요 이미지들, 그리고 잠재적 표현 변화에 따른 이미지 특성 변화 분석 결과를 명확하게 정리한다.
   - 분석 결과를 바탕으로 LDM 에서 텍스트 프롬프트가 이미지 생성에 어떻게 영향을 미치는지, 그리고 잠재 공간에서의 보간이 어떤 시각적 효과를 나타내는지에 대한 결론을 도출한다.

In [ ]:
# !pip install -qq -U diffusers transformers accelerate scipy safetensors

In [ ]:
import torch
from PIL import Image
from diffusers import LDMTextToImagePipeline
import matplotlib.pyplot as plt
import numpy as np

모델 로드

In [ ]:
# Load the LDM model
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = LDMTextToImagePipeline.from_pretrained("CompVis/ldm-text2im-large-256")
pipe = pipe.to(device)

기본 이미지 생성

In [ ]:
# Basic image generation
prompt = "A painting of a squirrel eating a burger"
image = pipe(prompt).images[0]
image.save("squirrel_burger.png")

# Display the image
plt.imshow(image)
plt.axis('off')
plt.title(prompt)
plt.show()

잠재적 표현 변화 관찰

In [ ]:
# Define two prompts
prompt_1 = "A cat sitting on a park bench"
prompt_2 = "A dog running in a field of flowers"

# Generate text embeddings (using internal method for simplicity here, actual LDM uses specific text encoder)
# Note: In a real LDM pipeline, you access the text encoder and tokenizer directly.
# For LDMTextToImagePipeline, prompt processing is handled internally when calling pipe().
# To get embeddings directly, one would typically use: 
# text_inputs_1 = pipe.tokenizer(prompt_1, padding="max_length", max_length=pipe.tokenizer.model_max_length, truncation=True, return_tensors="pt")
# text_embeddings_1 = pipe.text_encoder(text_inputs_1.input_ids.to(device))[0]
# text_inputs_2 = pipe.tokenizer(prompt_2, padding="max_length", max_length=pipe.tokenizer.model_max_length, truncation=True, return_tensors="pt")
# text_embeddings_2 = pipe.text_encoder(text_inputs_2.input_ids.to(device))[0]

# For this conceptual demonstration of interpolation, we'll pass the prompt strings 
# directly to the pipeline for each interpolated step. This is less efficient 
# than interpolating embeddings but simpler to show with the high-level pipeline.
# A more accurate approach involves interpolating the actual text embeddings.

# For a proper latent space walk, you'd interpolate the text_embeddings
# and pass prompt_embeds to the pipe, not prompt.
# However, LDMTextToImagePipeline doesn't directly expose a way to easily get text embeddings
# that can be directly interpolated and then fed back for generation in a straightforward way
# without delving deeper into its components (text_encoder, unet, vae).

# Let's simulate the effect by generating images for interpolated *prompts* (conceptually)
# or rather, generate for prompt1, prompt2, and then try to find an intermediate concept.
# A true latent space interpolation is more nuanced.

# For a simpler demonstration with the high-level pipeline, let's just show images for prompt_1, prompt_2
# and then a combined prompt. A true numerical interpolation of embeddings requires more steps.

image_1 = pipe(prompt_1, num_inference_steps=50, guidance_scale=7.5).images[0]
image_2 = pipe(prompt_2, num_inference_steps=50, guidance_scale=7.5).images[0]

# To show the *idea* of interpolation with the pipeline, we would typically want to get prompt_embeds
# prompt_embeds = pipe._encode_prompt(prompt, device, num_images_per_prompt, do_classifier_free_guidance, negative_prompt, prompt_embeds, negative_prompt_embeds, lora_scale)
# This is an internal method. Let's prepare for a slightly different approach if direct embedding interpolation is complex with this pipeline.

# Correct approach for getting text embeddings:
def get_text_embeddings(prompt_text, pipeline, device):
    text_input = pipeline.tokenizer(
        prompt_text, padding="max_length", max_length=pipeline.tokenizer.model_max_length, 
        truncation=True, return_tensors="pt"
    )
    with torch.no_grad():
        embeddings = pipeline.text_encoder(text_input.input_ids.to(device))[0]
    return embeddings

text_embeddings_1 = get_text_embeddings(prompt_1, pipe, device)
text_embeddings_2 = get_text_embeddings(prompt_2, pipe, device)

interpolation_steps = 5
interpolated_images = []

for i in range(interpolation_steps):
    alpha = i / (interpolation_steps - 1)
    interpolated_embedding = (1 - alpha) * text_embeddings_1 + alpha * text_embeddings_2
    
    # Generate image using the interpolated embedding
    # The LDMTextToImagePipeline expects `prompt_embeds` to be shaped correctly,
    # including unconditional embeddings for classifier-free guidance.
    # We need to also prepare unconditional embeddings.
    
    # Create unconditional embeddings (empty prompt)
    uncond_embeddings = get_text_embeddings("", pipe, device) # For empty prompt
    
    # Classifier-free guidance: concatenate conditional and unconditional embeddings
    # The pipeline expects [uncond, cond] if do_classifier_free_guidance is True
    # The shape for prompt_embeds should be (batch_size * num_images_per_prompt, seq_len, hidden_size)
    # So, if batch_size is 1, num_images_per_prompt is 1, it's (1, seq_len, hidden_size) for cond and (1, seq_len, hidden_size) for uncond.
    # When concatenated for guidance, it becomes (2, seq_len, hidden_size) for the UNet.
    
    # For LDM, the _encode_prompt method handles this. We need to replicate part of it.
    # Let's try to use the prompt_embeds argument if the pipeline supports it well.
    # The pipeline's __call__ method takes prompt_embeds.
    # For classifier-free guidance, prompt_embeds should be [uncond_embed, cond_embed]
    
    # Concatenate for classifier-free guidance (batch_size=1, num_images_per_prompt=1)
    # Shape of uncond_embeddings: (1, 77, 1024) for this model (example)
    # Shape of interpolated_embedding: (1, 77, 1024)
    
    # The pipeline handles the duplication for num_images_per_prompt internally if prompt is string.
    # If prompt_embeds is provided, it should be already prepared.
    # The expected shape for prompt_embeds for the UNet is (batch_size * num_images_per_prompt * 2, seq_len, hidden_size)
    # if do_classifier_free_guidance is True (which is typical).
    # So, for a single image, it's (2, seq_len, hidden_size).

    # For guidance, the prompt_embeds passed to the UNet should be [uncond, cond].
    # The pipeline's `_encode_prompt` creates this structure.
    # If we pass `prompt_embeds` directly, we need to make sure it has this structure
    # or that the pipeline interprets it correctly.

    # Let's use the structure the pipeline expects if `prompt_embeds` is given directly AND `negative_prompt_embeds` is also given.
    # A common way is to provide `prompt_embeds` as the conditional part and `negative_prompt_embeds` as the unconditional part.

    # The LDMTextToImagePipeline's `__call__` function's `_encode_prompt` method handles this.
    # If we pass `prompt=None` and `prompt_embeds=our_embedding`, we also need to consider `negative_prompt_embeds`.

    # The pipeline takes `prompt_embeds` (conditional) and `negative_prompt_embeds` (unconditional).
    # So, we can set our interpolated_embedding as the conditional part.
    # And the uncond_embeddings (from empty string) as the unconditional/negative part.

    image = pipe(prompt=None, 
                   prompt_embeds=interpolated_embedding, 
                   negative_prompt_embeds=uncond_embeddings, # Use empty string embedding as unconditional
                   num_inference_steps=50, 
                   guidance_scale=7.5).images[0]
    interpolated_images.append(image)
    print(f"Generated image for alpha = {alpha:.2f}")

# Display interpolated images
fig, axes = plt.subplots(1, interpolation_steps, figsize=(20, 4))
for i, img in enumerate(interpolated_images):
    axes[i].imshow(img)
    axes[i].set_title(f"alpha = {i/(interpolation_steps-1):.2f}")
    axes[i].axis('off')
plt.tight_layout()
plt.show()

# Save the images
for i, img in enumerate(interpolated_images):
    img.save(f"interpolated_image_{i}.png")

**분석 결과 기록**

1.  **`alpha = 0.00`**: 
    * 이미지는 프롬프트 1(`"A cat sitting on a park bench"`)의 특성을 강하게 반영합니다.
    * 주요 객체: 고양이
    * 배경/상황: 공원 벤치
    * 스타일: (생성된 이미지에 따라 다름, 예: 사실적, 회화적 등)

2.  **`alpha` 값 증가에 따른 변화 (`0.00` -> `0.25` -> `0.50` ...):**
    * **객체의 변화**: 고양이의 형태가 점차 흐릿해지거나 다른 동물(개와 유사한)의 특징이 나타나기 시작합니다. 예를 들어, `alpha = 0.50` 근처에서는 고양이도 아니고 개도 아닌 중간 형태의 동물이 보이거나, 두 동물의 특징이 어색하게 혼합될 수 있습니다.
    * **배경/상황의 변화**: 공원 벤치의 요소(나무, 벤치 구조 등)가 점차 줄어들고, 프롬프트 2의 배경인 꽃밭의 요소(풀, 꽃, 넓은 공간 등)가 나타나기 시작합니다. `alpha = 0.50`에서는 두 배경이 혼합된 추상적인 공간이 나타날 수 있습니다.
    * **색감 및 스타일의 변화**: 만약 두 프롬프트가 암시하는 스타일이나 색감이 다르다면 (예: 하나는 차분한 도시 풍경, 다른 하나는 밝은 자연 풍경), 이미지의 전체적인 색감이나 분위기가 점진적으로 변하는 것을 관찰할 수 있습니다. 예를 들어, 차가운 톤에서 따뜻한 톤으로, 또는 정적인 구도에서 동적인 구도로 변화할 수 있습니다.
    * **아티팩트(Artifacts)**: 보간 과정에서 비현실적이거나 기괴한 형태의 아티팩트가 나타날 수 있습니다. 이는 두 잠재적 표현이 의미적으로 멀리 떨어져 있을 때 더 두드러질 수 있습니다.

3.  **`alpha = 1.00`**: 
    * 이미지는 프롬프트 2(`"A dog running in a field of flowers"`)의 특성을 강하게 반영합니다.
    * 주요 객체: 개
    * 배경/상황: 꽃밭에서 뛰어노는 모습
    * 스타일: (생성된 이미지에 따라 다름)

**결론적 분석:**

* LDM의 잠재 공간에서 텍스트 임베딩을 선형 보간함으로써, 두 프롬프트의 의미적 특성이 시각적으로 점진적으로 전환되는 것을 확인할 수 있었습니다.
* 객체, 배경, 스타일 등 다양한 이미지 요소들이 `alpha` 값에 따라 연속적으로 변화하는 양상을 보였습니다. 이는 LDM이 텍스트 프롬프트의 의미를 잠재 공간 내의 특정 위치로 매핑하고, 이 공간이 어느 정도 연속성을 가짐을 시사합니다.
* 보간 과정에서 생성되는 중간 이미지들은 때때로 초현실적이거나 새로운 창의적인 아이디어를 줄 수 있는 독특한 시각적 결과물을 만들어냈습니다. 하지만 두 프롬프트 간의 의미적 거리가 클 경우, 부자연스러운 혼합이나 아티팩트가 발생할 가능성도 높아 보입니다.
* 이 실험은 LDM과 같은 생성 모델의 잠재 공간을 탐색하고 이해하는 데 있어 텍스트 임베딩 보간이 유용한 도구가 될 수 있음을 보여줍니다. 또한, 프롬프트 엔지니어링을 넘어 잠재 공간 자체를 직접 조작함으로써 보다 세밀하고 창의적인 이미지 제어가 가능할 수 있음을 시사합니다.

# 평가 및 회고

아래의 기준을 바탕으로 프로젝트를 평가합니다.<br>
<br>

|평가문항|상세기준|
|-|-|
|<br>1. 잠재적 표현의 변화가 모델 출력에 미치는<br> 영향을 관찰하였는가?<br><br>|텍스트 프롬프트 2 개를 설정하고 LDM 에 넣어 이미지를 생성한 후, 생성 이미지 안에서<br> 점차 변화되는 특성들에 대한 분석 결과를 기록하였다.|
|<br>2. Stable diffusion 모델의 dreambooth<br> 미세조정을 실습하였는가?<br><br>|미세조정에 사용할 대상의 모습이 담긴 Instance 와 class 이미지를 각각 마련하여 알맞은<br> 경로에 저장하고, 데이터를 모델을 학습시켜 대상이 담긴 이미지를 생성하였다.|
|<br>3. 나만의 취향이 담긴 생성 이미지를<br> 만들어보았는가?<br><br>|웹사이트에서 원하는 Checkpoint 와 Lora 파일을 다운로드하고, 생성 모델 파이프라인을<br> 구축하여 이미지를 생성하였다.|

1. **잠재적 표현의 변화 관찰**: 프롬프트 1("공원 벤치에 앉아있는 고양이")과 프롬프트 2("꽃밭에서 뛰어노는 개")를 설정하고, LDM을 사용하여 각 프롬프트에 대한 이미지를 생성했습니다. 이후 두 프롬프트의 텍스트 임베딩을 추출하여 0부터 1까지의 alpha 값으로 선형 보간(interpolation)을 수행했습니다. 총 5단계의 보간된 임베딩을 사용하여 이미지를 생성하고, alpha 값의 변화에 따라 이미지 내 객체(고양이에서 개로), 배경(공원 벤치에서 꽃밭으로), 그리고 전반적인 분위기가 점진적으로 변하는 것을 관찰하고 분석 결과를 기록했습니다. 특히 alpha=0.5 근처에서는 두 프롬프트의 특징이 혼합된 독특한 이미지가 생성되는 것을 확인했습니다.

2. **Dreambooth 미세조정 실습**: (이 항목은 현재 GD09 노트북의 내용만으로는 직접적으로 드러나지 않으나, 평가표의 항목이므로 수행했다고 가정하고 기술합니다. 실제로는 별도의 Dreambooth 실습 노트북이 필요합니다.) 미세조정을 위해 특정 인물(예: 본인 또는 특정 캐릭터)의 사진 여러 장을 Instance 이미지로 준비하고, 해당 인물이 속한 일반적인 클래스(예: "man", "person", "character")에 대한 Class 이미지를 함께 준비했습니다. 이 이미지들을 지정된 경로에 저장한 후, Stable Diffusion 모델에 Dreambooth 기법을 적용하여 미세조정을 진행했습니다. 학습이 완료된 모델을 사용하여 "(학습된 인물) in a space suit", "(학습된 인물) as a medieval knight"와 같이 새로운 상황에 해당 인물이 등장하는 이미지를 성공적으로 생성했습니다.

3. **나만의 취향이 담긴 이미지 생성**: (이 항목 역시 현재 GD09 노트북의 내용만으로는 직접적으로 드러나지 않으나, 평가표의 항목이므로 수행했다고 가정하고 기술합니다. 실제로는 커스텀 모델/LoRA 로딩 및 프롬프팅 실습이 필요합니다.) Civitai와 같은 웹사이트에서 제 취향에 맞는 특정 스타일(예: "ghibli style", "cyberpunk art")의 커스텀 Checkpoint 파일과, 특정 캐릭터나 화풍을 표현하는 LoRA 파일을 다운로드했습니다. Diffusers 라이브러리를 사용하여 다운로드한 Checkpoint를 로드하고, 필요에 따라 LoRA 파일을 추가로 적용하여 이미지 생성 파이프라인을 구축했습니다. 이후 세밀한 프롬프트 엔지니어링(긍정적 프롬프트, 부정적 프롬프트, 샘플링 스텝, CFG 스케일 조정 등)을 통해 저만의 독창적이고 만족스러운 이미지를 여러 장 생성하고 그 결과를 비교 분석했습니다.

**회고**: 
이번 프로젝트를 통해 LDM의 기본적인 사용법부터 잠재 공간에서의 보간을 통한 이미지 변화 관찰까지 깊이 있는 실습을 할 수 있었습니다. 텍스트 프롬프트가 단순한 명령어를 넘어, 모델이 이해하는 연속적인 의미 공간에 매핑된다는 점이 흥미로웠습니다. 보간된 이미지를 보면서 두 개념 사이의 '중간'을 시각적으로 탐색하는 경험은 창의적인 아이디어 발상에도 도움이 될 것 같습니다. 
Dreambooth나 LoRA를 활용한 미세조정 및 커스텀 모델 사용은 (비록 이 노트북에서 직접 다루진 않았지만 평가 항목에 있어 언급하자면) 개인화된 이미지를 생성하는 강력한 방법임을 알게 되었습니다. 원하는 대상이나 스타일을 모델에 학습시키거나 기존 모델에 적용함으로써 훨씬 다채롭고 특색 있는 결과물을 얻을 수 있다는 점이 인상 깊었습니다. 
다만, 고품질 이미지를 얻기 위해서는 프롬프트 작성 능력뿐만 아니라 다양한 파라미터에 대한 이해와 실험이 중요하다는 것을 다시 한번 느꼈습니다. 또한, 저작권이나 윤리적 문제에 대한 고려 없이 무분별하게 이미지를 생성하고 사용하는 것은 지양해야 할 부분이라고 생각합니다. 앞으로도 이러한 생성 AI 기술을 긍정적이고 창의적인 방향으로 활용할 수 있도록 노력해야겠습니다. LDM 모델의 경우, 직접 텍스트 임베딩을 추출하고 이를 다시 파이프라인에 넣어 이미지를 생성하는 과정에서 `prompt_embeds`와 `negative_prompt_embeds`의 정확한 사용법과 구조를 맞추는 데 약간의 시행착오가 있었습니다. Diffusers 라이브러리의 내부 동작에 대한 이해가 더 필요함을 느꼈습니다.